Открытие гугл диска

In [ ]:
# # !rm -rf /root/.config/Google/DriveFS
# !rm -rf /root/.config/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import os.path
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import random
from torchvision import transforms

Дата лоадер


In [ ]:
# def get_data(batch_size,image_folder= '/content/drive/MyDrive/00/image_0', movements_file = '/content/drive/MyDrive/00/00.txt'):
#     # image_folder = '/content/drive/MyDrive/00/image_0' # замените на путь к вашей папке

#     def get_images(image_folder):
#         image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith('.png')]
#         image_files.sort()

#         images = []
#         for file_path in image_files:
#             try:
#                 img = Image.open(file_path).convert('L')

#                 img_array = np.array(img, dtype=np.float32) / 255.0  # Преобразуем изображение в массив numpy
#                 images.append(img_array)

#             except Exception as e:
#                 print(f"Ошибка при загрузке {file_path}: {e}")

#         images = np.array(images) # преобразуем в numpy массив
#         print(images.shape) # проверяем размерность массива
#         return images

#     images = get_images(image_folder)

#     def create_movements(movements_file):
#         movements = []
#         with open(movements_file, 'r') as f:
#             for line in f:
#                 data = line.strip().split()
#                 movements.append([float(x) for x in data])
#         return movements

#     movements = create_movements(movements_file)


#     def transform_mov(movements):
#         new_movements =  []
#         for matrix in movements:
#             new_matrix = [
#                 matrix[0:4].tolist(),
#                 matrix[4:8].tolist(),
#                 matrix[8:12].tolist(),
#                 [0,0,0,1]
#                 ]
#             new_matrix = torch.tensor(new_matrix)
#             new_movements.append(new_matrix)

#             res_movements = []

#         for i in range(len(new_movements)-1):
#             T1_inv = torch.linalg.inv(new_movements[i])
#             T_rel = torch.matmul(new_movements[i+1], T1_inv)
#             res_matrix = T_rel[:3]
#             res_matrix = torch.cat([ res_matrix[0],  res_matrix[1],  res_matrix[2]])
#             res_movements.append(res_matrix)

#         return res_movements

#     movements = transform_mov(movements)


#     class ImageDataset(Dataset):
#         def __init__(self,images, movements):
#             self.images = images
#             self.movements = movements

#         def __len__(self):
#             return len(self.movements)

#         def __getitem__(self,idx):
#             image1 = torch.tensor(self.images[idx],dtype = torch.float32).unsqueeze(0)
#             image2 = torch.tensor(self.images[idx + 1],dtype = torch.float32).unsqueeze(0)
#             movement = torch.tensor(self.movements[idx], dtype = torch.float32)
#             return (image1,image2), movement

#     train_size = int(0.9*len(images))
#     train_dataset = ImageDataset(images[:train_size], movements[:train_size])
#     validation_dataset = ImageDataset(images[train_size:], movements[train_size:])

#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
#     validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle = True)

#     return train_loader,validation_loader


In [ ]:
def get_data(batch_size, image_folder='/content/drive/MyDrive/00/image_0', movements_file='/content/drive/MyDrive/00/00.txt'):
    # Чтение движений из файла
    def create_movements(movements_file):
        movements = []
        with open(movements_file, 'r') as f:
            for line in f:
                data = list(map(float, line.strip().split()))
                movements.append(data)
        return movements

    movements = create_movements(movements_file)

    # Преобразование движений
    # def transform_mov(movements):
    #     new_movements = []
    #     for matrix in movements:
    #         new_matrix = [
    #             matrix[0:4],
    #             matrix[4:8],
    #             matrix[8:12],
    #             [0, 0, 0, 1]
    #         ]
    #         new_matrix = torch.tensor(new_matrix, dtype=torch.float32)
    #         new_movements.append(new_matrix)

    #     res_movements = []
    #     for i in range(len(new_movements) - 1):
    #         T1_inv = torch.linalg.inv(new_movements[i])
    #         T_rel = torch.matmul(new_movements[i + 1], T1_inv)
    #         res_matrix = T_rel[:3]
    #         res_matrix = torch.cat([res_matrix[0], res_matrix[1], res_matrix[2]])
    #         res_movements.append(res_matrix)


    #     return res_movements

    def transform_mov(movements):
        res_movements = []
        for i in range(len(movements)-1):
            m1 = torch.tensor(movements[i])
            m2 = torch.tensor(movements[i+1])
            res = m2-m1
            distance = math.sqrt(pow(res[3],2) + pow(res[7],2) + pow(res[11],2))
            res_movements.append([distance ])
        return res_movements

    movements = transform_mov(movements)

    # Оптимизированный класс Dataset
    class ImageDataset(Dataset):
        def __init__(self, image_folder, movements,file_num1,file_num2, transform=None ):
            # Получаем список всех файлов .png и сортируем их
            all_image_files = sorted([os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith('.png')])

            # Ограничиваем количество файлов, если передан file_num
            if file_num1 and file_num2:
                all_image_files = all_image_files[file_num1:file_num2]

            self.image_files = all_image_files
            self.movements = movements
            self.transform = transform

        def __len__(self):
            # Возвращаем минимальную длину между движениями и изображениями - 1
            return min(len(self.movements), len(self.image_files) - 1)


        def __getitem__(self, idx):
            image1_path = self.image_files[idx]
            image2_path = self.image_files[idx + 1]

            image1 = Image.open(image1_path).convert('L')
            image2 = Image.open(image2_path).convert('L')

            if self.transform:
                image1 = self.transform(image1)
                image2 = self.transform(image2)

            movement = torch.tensor(self.movements[idx], dtype=torch.float32)
            return (image1, image2), movement

    # Трансформации для уменьшения использования памяти
    transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.ToTensor()
    ])

    # Разделение на тренировочную и валидационную выборки
    train_size = int(0.9 * len(movements))
    train_size = 4000
    train_dataset = ImageDataset(image_folder, movements[:train_size], 0, train_size, transform=transform)
    validation_dataset = ImageDataset(image_folder, movements[train_size:],train_size, len(movements), transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
    test_loader = None
    return train_loader, validation_loader,test_loader


In [ ]:
import math

In [ ]:
def get_pathes(base_folder ):
    paths = []
    for i in range(11):
        folder_name = f'{i:02d}'
        image_folder = os.path.join(base_folder,folder_name,'image_0')
        movements_file = os.path.join(base_folder,folder_name,f'{folder_name}.txt')
        paths.append((image_folder,movements_file))
    return paths

def get_data(batch_size, base_folder = '/content/drive/MyDrive'):
    paths = get_pathes(base_folder)

    # Выбор папок для тренировочных, валидационных и тестовых данных
    validation_folder = paths[-2]  # Предпоследняя папка
    test_folder = paths[-1]        # Последняя папка
    train_folders = paths[:-2]     # Все остальные папки

    # Чтение движений из файла
    def create_movements(movements_file):
        movements = []
        with open(movements_file, 'r') as f:
            for line in f:
                data = list(map(float, line.strip().split()))
                movements.append(data)
        return movements

    # movements = create_movements(movements_file)

    # Преобразование движений
    # def transform_mov(movements):
    #     new_movements = []
    #     for matrix in movements:
    #         new_matrix = [
    #             matrix[0:4],
    #             matrix[4:8],
    #             matrix[8:12],
    #             [0, 0, 0, 1]
    #         ]
    #         new_matrix = torch.tensor(new_matrix, dtype=torch.float32)
    #         new_movements.append(new_matrix)

    #     res_movements = []
    #     for i in range(len(new_movements) - 1):
    #         T1_inv = torch.linalg.inv(new_movements[i])
    #         T_rel = torch.matmul(new_movements[i + 1], T1_inv)
    #         res_matrix = T_rel[:3]
    #         res_matrix = torch.cat([res_matrix[0], res_matrix[1], res_matrix[2]])
    #         res_movements.append(res_matrix)


    #     return res_movements

    # def transform_mov(movements):
    #     res_movements = []
    #     for i in range(len(movements)-1):
    #         m1 = torch.tensor(movements[i])
    #         m2 = torch.tensor(movements[i+1])
    #         res = m2-m1
    #         res_movements.append(res)
    #     return res_movements

    def transform_mov(movements):
        res_movements = []
        for i in range(len(movements)-1):
            m1 = torch.tensor(movements[i])
            m2 = torch.tensor(movements[i+1])
            res = m2-m1
            distance = math.sqrt(pow(res[3],2) + pow(res[7],2) + pow(res[11],2))
            res_movements.append([distance ])
        return res_movements



    # movements = transform_mov(movements)

    # Оптимизированный класс Dataset
    class ImageDataset(Dataset):
        def __init__(self, image_folder, movements, transform=None ):
            # Получаем список всех файлов .png и сортируем их
            all_image_files = sorted([os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith('.png')])
            self.image_files = all_image_files
            self.movements = movements
            self.transform = transform

        def __len__(self):
            # Возвращаем минимальную длину между движениями и изображениями - 1
            return min(len(self.movements), len(self.image_files) - 1)


        def __getitem__(self, idx):
            image1_path = self.image_files[idx]
            image2_path = self.image_files[idx + 1]

            image1 = Image.open(image1_path).convert('L')
            image2 = Image.open(image2_path).convert('L')

            if self.transform:
                image1 = self.transform(image1)
                image2 = self.transform(image2)

            movement = ( self.movements[idx].clone().detach().float()
                        if isinstance(self.movements[idx], torch.Tensor)
                        else torch.tensor(self.movements[idx], dtype=torch.float32) )
            return (image1, image2), movement


    # Трансформации для уменьшения использования памяти
    transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((376,1240)),
        transforms.ToTensor()
    ])

    def load_dataset(folders):
        images, movements = [], []
        for image_folder, movements_file in folders:
            mov = create_movements(movements_file)
            mov = transform_mov(mov)
            movements.append(mov)
            images.append(image_folder)
        return images, movements

    train_images, train_movements = load_dataset(train_folders)
    train_dataset = []
    for index,image_folder in enumerate(train_images):
        dataset = ImageDataset(image_folder, train_movements[index], transform=transform)
        train_dataset.append(dataset)
    train_dataset = torch.utils.data.ConcatDataset(train_dataset)

    # Загрузка валидационных данных
    val_images, val_movements = load_dataset([validation_folder])
    val_dataset = ImageDataset(val_images[0], val_movements[0], transform=transform)


   # Загрузка тестовых данных
    test_images, test_movements = load_dataset([test_folder])
    test_dataset = ImageDataset(test_images[0], test_movements[0], transform=transform)

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True, num_workers=4, pin_memory=True)
    validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = False, num_workers=4, pin_memory=True)

    return train_loader, validation_loader, test_loader




In [ ]:
# train_loader, validation_loader = get_data(4)
# print(type(train_loader))
# (image1,image2),movements = next(iter(train_loader))
# print(image1.shape,)
# print(image2.shape)
# print(movements.shape)
# print(type(movements))
# print(movements)
# # print(train_loader.shape)


<class 'torch.utils.data.dataloader.DataLoader'>


<ipython-input-11-9a000deb2846>:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  movement = torch.tensor(self.movements[idx], dtype=torch.float32)


torch.Size([4, 1, 376, 1241])
torch.Size([4, 1, 376, 1241])
torch.Size([4, 12])
<class 'torch.Tensor'>
tensor([[ 9.9885e-01,  3.4306e-03, -4.7780e-02,  1.0794e+01, -3.3760e-03,
          9.9999e-01,  1.2231e-03, -6.5439e-01,  4.7784e-02, -1.0604e-03,
          9.9886e-01,  5.6170e+00],
        [ 9.9991e-01, -8.3254e-03, -1.0746e-02,  5.6306e+00,  8.3262e-03,
          9.9997e-01,  2.5423e-05, -1.2226e+00,  1.0745e-02, -1.1488e-04,
          9.9994e-01, -1.2722e+00],
        [ 9.9995e-01,  8.3675e-03, -4.9610e-03,  2.2081e+00, -8.3621e-03,
          9.9996e-01,  1.1193e-03, -3.9281e-02,  4.9702e-03, -1.0778e-03,
          9.9999e-01,  3.4265e-01],
        [ 9.9999e-01, -4.4478e-03,  3.0920e-03, -1.7784e-01,  4.4483e-03,
          9.9999e-01, -1.6031e-04, -2.7174e-01, -3.0909e-03,  1.7405e-04,
          1.0000e+00,  8.1690e-01]])


In [ ]:

# movements_file='/content/drive/MyDrive/00/00.txt'
# # Чтение движений из файла
# def create_movements(movements_file):
#     movements = []
#     with open(movements_file, 'r') as f:
#         for line in f:
#             data = list(map(float, line.strip().split()))
#             movements.append(data)
#     return movements

# movements = create_movements(movements_file)
# print("start movement after opening and spliting ",movements[:2])

# # Преобразование движений
# def transform_mov(movements):
#     new_movements = []
#     for matrix in movements:
#         new_matrix = [
#             matrix[0:4],
#             matrix[4:8],
#             matrix[8:12],
#             [0, 0, 0, 1]
#         ]
#         new_matrix = torch.tensor(new_matrix, dtype=torch.float32)
#         new_movements.append(new_matrix)
#     print("после преобразование в матрицу 4 на 4 и добавление одной строки ", new_movements[:2])

#     res_movements = []
#     # for i in range(len(new_movements) - 1):
#     for i in range(2):
#         T1_inv = torch.linalg.inv(new_movements[i])
#         T_rel = torch.matmul(new_movements[i + 1], T1_inv)
#         res_matrix = T_rel[:3]
#         res_matrix = torch.cat([res_matrix[0], res_matrix[1], res_matrix[2]])
#         print("после преобразование матриці ", res_matrix)
#         res_movements.append(res_matrix)
#         print("добавление в один массив ", res_movements)

#     return res_movements

# movements = transform_mov(movements)


EVALUATION нужно еще функцию написать для подсчета точности или потерь вот тут внизу

In [ ]:
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


In [ ]:
def compute_accuracy(model, data_loader, device):
    model.eval()
    total_loss = 0.0
    num_batches = 0
    correct_predictions = 0   # Счетчик корректных предсказаний
    total_samples = 0         # Общее количество элементов

    criterion = torch.nn.MSELoss()

    with torch.no_grad():
        for batch_idx, ((feature1, feature2),targets) in enumerate(data_loader):
            feature1 = feature1.to(device)
            feature2 = feature2.to(device)
            targets = targets.to(device)

            predicted_y = model(feature1, feature2)

            # Вычисление ошибки
            loss = criterion(predicted_y, targets)
            total_loss += loss.item()
            num_batches += 1

            # Проверка критерия качества
            relative_difference = torch.abs((predicted_y - targets) / targets)  # Расчет относительного отклонения
            valid_predictions = torch.all(relative_difference <= 0.1, dim=1)    # Условие: отклонение <= 10%
            correct_predictions += torch.sum(valid_predictions).item()
            total_samples = total_samples + targets.size(0)


    avg_loss = total_loss / num_batches             # Расчет средней ошибки
    accuracy = correct_predictions / total_samples  # Расчет доли верных предсказаний
    return avg_loss, accuracy

Model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

model 1

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 4, stride = 2),# 187x620
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # nn.MaxPool2d(2, 2),  # 93x310
            nn.Dropout(0.25),

            nn.Conv2d(32, 32, kernel_size = 4, stride = 2),# 93x110
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # nn.Dropout(0.25),

            nn.Conv2d(32, 64, kernel_size = 4, stride = 2),#45x154
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # nn.MaxPool2d(2, 2),  # 22x77
            # nn.Dropout(0.25),

            nn.Conv2d(64, 64, kernel_size = 4, stride = 2),# 93x110
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size = 4, stride = 2),# 10x37
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # nn.MaxPool2d(2, 2),  # 5x18
            # nn.Dropout(0.25),

            nn.Conv2d(128, 128, kernel_size = 4, stride = 2),# 10x37
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # nn.Dropout(0.25),

            # nn.Conv2d(128, 256, kernel_size = 4,step = 2),
            # nn.BatchNorm2d(256),
            # nn.ReLU(),
            nn.AdaptiveAvgPool2d((5, 5))  # Глобальный пуллинг к фиксированному размеру
        )
        self.lstm1 = nn.LSTMCell(input_size=128 * 5 * 5, hidden_size=256)
        self.lstm2 = nn.LSTMCell(input_size=128 * 5 * 5, hidden_size=256)

        self.network = nn.Sequential(
            # nn.Linear(2 * 128 * 5 * 5, 128),
            # nn.ReLU(),
            # nn.Dropout(p=0.5),
            # nn.Linear(128, 64),
            # nn.ReLU(),
            # nn.Dropout(p=0.5),
            # nn.Linear(64, 12),
            nn.Linear(256, 1)
        )
        # self.network = nn.Linear(2*128*5*5,12)


    def forward(self,image1,image2):
        out_features1 = self.features(image1)
        out_features2 = self.features(image2)

        out_features1 = torch.flatten(out_features1, 1) # Flatten all dimensions except batch
        out_features2 = torch.flatten(out_features2, 1)

        batch_size = out_features1.size(0)

        # Инициализируем скрытые состояния и состояния ячеек для lstm1
        h_t1 = torch.zeros(batch_size, self.lstm1.hidden_size, device=out_features1.device)
        c_t1 = torch.zeros(batch_size, self.lstm1.hidden_size, device=out_features1.device)

        # Пропускаем признаки первой картинки через lstm1 (один временной шаг)
        h_t1_next, c_t1_next = self.lstm1(out_features1, (h_t1, c_t1))

        # Пропускаем признаки второй картинки через lstm2 (один временной шаг)
        h_t2_next, c_t2_next = self.lstm2(out_features2, (h_t1_next, c_t1_next))

        # Конкатенируем выходные скрытые состояния от обоих LSTMCell
        # out = torch.cat((h_t1_next, h_t2_next), dim=1) # [batch_size, 256]

        out = self.network(h_t2_next) # [batch_size, 1]
        return out


        # out_features = torch.cat((out_features1,out_features2),dim = 1 )
        # out = self.network(out_features)
        # return out

Alternative Model 1


In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(2, 32, kernel_size = 5, stride = 2),# 187x620
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # nn.MaxPool2d(2, 2),  # 93x310
            # nn.Dropout(0.25),

            nn.Conv2d(32, 32, kernel_size= 4, stride= 2, groups= 32),# 93x310
            nn.Conv2d(32, 32, kernel_size= 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # nn.Dropout(0.25),

            nn.Conv2d(32, 64, kernel_size = 4, stride = 2),#45x155
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # nn.MaxPool2d(2, 2),  # 22x77
            # nn.Dropout(0.25),

            nn.Conv2d(64, 64, kernel_size= 4, stride= 2, groups= 64),# 22x77
            nn.Conv2d(64, 64, kernel_size= 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size = 4, stride = 2),# 11x38
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # nn.MaxPool2d(2, 2),  # 5x18
            # nn.Dropout(0.25),

            nn.Conv2d(128, 128, kernel_size = 4, stride = 2, groups= 128),# 5x19
            nn.Conv2d(128, 128, kernel_size= 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # nn.Dropout(0.25),

            # nn.Conv2d(128, 256, kernel_size = 4,step = 2),
            # nn.BatchNorm2d(256),
            # nn.ReLU(),
            # nn.AdaptiveAvgPool2d((5, 5))  # Глобальный пуллинг к фиксированному размеру
        )
        # self.lstm1 = nn.LSTMCell(input_size=128 * 5 * 5, hidden_size=256)
        # self.lstm2 = nn.LSTMCell(input_size=128 * 5 * 5, hidden_size=256)

        self.network = nn.Sequential(
            nn.Linear(6528, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        # self.network = nn.Linear(2*128*5*5,12)


    def forward(self,image1,image2):
        # print('__1')
        images = torch.cat((image1, image2), dim=1)
        # print('images shape  ', images.shape)
        out_features = self.features(images)
        # print('len out_features  ', out_features.shape)
        out_features = torch.flatten(out_features, 1)
        # print('float out_features  ', out_features.shape)
        out = self.network(out_features)
        # print('__3')
        return out


        # out_features = torch.cat((out_features1,out_features2),dim = 1 )
        # out = self.network(out_features)
        # return out

EfficientNetV2


In [ ]:
class MBConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, expansion_factor, stride, kernel_size=3):
        super(MBConvBlock, self).__init__()

        # Слой расширения (expansion)
        self.expand = nn.Conv2d(in_channels, in_channels * expansion_factor, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(in_channels * expansion_factor)

        # Depthwise свертка
        self.depthwise = nn.Conv2d(in_channels * expansion_factor, in_channels * expansion_factor,
                                    kernel_size = kernel_size, stride = stride,
                                   padding = kernel_size//2, groups = in_channels * expansion_factor)
        self.bn2 = nn.BatchNorm2d(in_channels * expansion_factor)

        # Сжатие (projection) и выходной слой
        self.project = nn.Conv2d(in_channels * expansion_factor, out_channels, kernel_size=1)
        self.bn3 = nn.BatchNorm2d(out_channels)

        self.use_residual = (in_channels == out_channels) and (stride == 1)

    def forward(self, x):
        identity = x

        x = F.relu(self.bn1(self.expand(x)))
        x = F.relu(self.bn2(self.depthwise(x)))
        x = self.bn3(self.project(x))

        if self.use_residual:
            x += identity

        return x


class Network(nn.Module):
    def __init__(self, num_classes=1):
        super(Network, self).__init__()

        # Первый сверточный слой
        self.conv_stem = nn.Conv2d(2, 32, kernel_size= (6, 10), stride= (2, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        # Последовательность MBConv блоков
        self.blocks = nn.Sequential(
            MBConvBlock(32, 16, expansion_factor=1, stride=1),
            MBConvBlock(16, 24, expansion_factor=6, stride=2),
            MBConvBlock(24, 40, expansion_factor=6, stride=2),
            MBConvBlock(40, 80, expansion_factor=6, stride=2),
            MBConvBlock(80, 112, expansion_factor=6, stride=1),
            MBConvBlock(112, 192, expansion_factor=6, stride=2),
            MBConvBlock(192, 320, expansion_factor=6, stride=1)
        )

        # Глобальный Average Pooling
        self.global_pool = nn.AdaptiveAvgPool2d(1)

        # Полносвязный слой для классификации
        self.fc = nn.Linear(320, num_classes)

    def forward(self, image1,image2):
        x = torch.cat((image1, image2), dim=1)
        x = F.relu(self.bn1(self.conv_stem(x)))
        x = self.blocks(x)
        x = self.global_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


EfficientNetV2  version 2


In [ ]:
class MBConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, expansion_factor, stride, kernel_size=3):
        super(MBConvBlock, self).__init__()

        self.use_residual = (in_channels == out_channels) and (stride == 1)
        hidden_dim = in_channels * expansion_factor

        self.conv = nn.Sequential(
            # Слой расширения (expansion)
            nn.Conv2d(in_channels, hidden_dim, kernel_size=1, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),
            # Depthwise свертка
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=kernel_size, stride=stride,
                      padding=kernel_size // 2, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),
            # Сжатие (projection)
            nn.Conv2d(hidden_dim, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
        )

    def forward(self, x):
        if self.use_residual:
            return x + self.conv(x)
        else:
            return self.conv(x)

class Network(nn.Module):
    def __init__(self, num_classes=1):
        super(Network, self).__init__()

         # Первый сверточный слой с измененными параметрами
        self.conv_stem = nn.Conv2d(2, 32, kernel_size=(6, 10), stride=(2, 3), padding=1)# 187, 411
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU6(inplace=True)

        # Последовательность MBConv блоков
        self.blocks = nn.Sequential(
            MBConvBlock(32, 16, expansion_factor=1, stride=1), #187, 411
            MBConvBlock(16, 24, expansion_factor=6, stride=2), # 94, 206
            MBConvBlock(24, 40, expansion_factor=6, stride=2),# 47, 103
            MBConvBlock(40, 80, expansion_factor=6, stride=2),# 24, 52
            MBConvBlock(80, 112, expansion_factor=6, stride=1),# 24, 52
            MBConvBlock(112, 192, expansion_factor=6, stride=2),# 12, 26
            MBConvBlock(192, 320, expansion_factor=6, stride=1) # 12, 26
        )

        # Последний сверточный слой
        self.conv_head = nn.Conv2d(320, 1280, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(1280)
        self.relu2 = nn.ReLU6(inplace=True)

        # Глобальный Average Pooling
        self.global_pool = nn.AdaptiveAvgPool2d(1)

        # Полносвязный слой для классификации
        self.fc = nn.Linear(1280, num_classes)

        self._initialize_weights()

    def forward(self, image1, image2):
        x = torch.cat((image1, image2), dim=1)
        x = self.relu1(self.bn1(self.conv_stem(x)))
        x = self.blocks(x)
        x = self.relu2(self.bn2(self.conv_head(x)))
        x = self.global_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

In [ ]:
directory_path = '/content/drive/My Drive/checpoint/model_3_5/'

# Проверка существования директории
if not os.path.exists(directory_path):
    # Создание директории
    os.makedirs(directory_path)
    print(f"Директория {directory_path} была создана.")
else:
    print(f"Директория {directory_path} уже существует.")

Директория /content/drive/My Drive/checpoint/model_3_5/ была создана.


Описание модели, оптимизатора, шедулера

In [ ]:
txt_model_opt_shed = '''

  model = Network()
  model = model.to(DEVICE)
  optimizer = torch.optim.Adam(model.parameters(),lr = 0.001, weight_decay=0.01)
  train_loader, validation_loader,test_loader = get_data(batch_size = BATCH_SIZE)
  scheduler = StepLR(optimizer, step_size=2, gamma=0.3)
  def count_parameters(model):
      return sum(p.numel() for p in model.parameters() if p.requires_grad)

  print(f'The model has {count_parameters(model):,} trainable parameters')
'''

Создание словаря для сохранения результатов

In [ ]:

checkpoint_dict = {
    'model_description' : txt_model_opt_shed,
    'state_model' : None,
    'state_opt'   : None,
    'state_scheduler' : None,

    'train_loss' :None,
    'val_loss'   : None,
    'best_loss'  : None,
    'train_acc'  : None,
    'val_acc'    : None,

    'EPOCHS'     : None,
    'current_epoch' : None,
    'learning' : None

}

EfficientNetV2 version 3

In [ ]:
class MBConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, expansion_factor, stride, kernel_size=3):
        super(MBConvBlock, self).__init__()

        self.use_residual = (in_channels == out_channels) and (stride == 1)
        hidden_dim = in_channels * expansion_factor

        self.conv = nn.Sequential(
            # Слой расширения (expansion)
            nn.Conv2d(in_channels, hidden_dim, kernel_size=1),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),

            # Depthwise свертка
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=kernel_size, stride=stride,
                      padding=kernel_size // 2, groups=hidden_dim),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),
            nn.Dropout2d(p= 0.2),

            # Сжатие (projection)
            nn.Conv2d(hidden_dim, out_channels, kernel_size=1),
            nn.BatchNorm2d(out_channels),

        )

    def forward(self, x):
        if self.use_residual:
            return x + self.conv(x)
            print(self.conv(x).shape)
        else:
            return self.conv(x)
            print(self.conv(x).shape)

class Network(nn.Module):
    def __init__(self, num_classes=1):
        super(Network, self).__init__()

        # Первый сверточный слой с измененными параметрами
        self.conv_stem = nn.Conv2d(2, 32, kernel_size=(6, 10), stride=(2, 3), padding=1)# 187, 411
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU6(inplace=True)

        self.blocks = nn.Sequential(
            # MBConvBlock(32, 64, expansion_factor=1, stride=2), #
            MBConvBlock(32, 48, expansion_factor=6, stride=2),# 93, 206
            MBConvBlock(48, 64, expansion_factor=6, stride=2),# 47, 103
            MBConvBlock(64, 64, expansion_factor=6, stride=1), # 47, 103
            MBConvBlock(64, 128, expansion_factor=6, stride=2),# 24, 52
            MBConvBlock(128, 128, expansion_factor=6, stride=1),# 24, 52
            MBConvBlock(128, 256, expansion_factor=6, stride=2),# 12, 26
            MBConvBlock(256, 256, expansion_factor=6, stride=1),# 12, 26
            MBConvBlock(256, 512, expansion_factor=6, stride=2), # 6, 13
            MBConvBlock(512, 512, expansion_factor=6, stride=2) # 3, 6

        )

        # # Глобальный Average Pooling
        # self.global_pool = nn.AdaptiveAvgPool2d(1)

        # Полносвязный слой для классификации
        self.fc1 = nn.Linear(512*3*7, 16)
        self.relu_fc = nn.ReLU(inplace=True)
        self.dropout_fc = nn.Dropout(p= 0.2) # Dropout после первого FC слоя
        self.fc2 = nn.Linear(16, 1)

        self._initialize_weights()

    def forward(self, image1, image2):
        x = torch.cat((image1, image2), dim=1)
        x = self.relu1(self.bn1(self.conv_stem(x)))
        x = self.blocks(x)
        # x = self.relu2(self.bn2(self.conv_head(x)))
        # x = self.global_pool(x)
        x = torch.flatten(x, 1)
        x = self.dropout_fc(self.relu_fc(self.fc1(x)))
        x = self.fc2(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

EfficientNetV2 version 4

In [ ]:
class MBConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, expansion_factor, stride, kernel_size=3):
        super(MBConvBlock, self).__init__()

        self.use_residual = (in_channels == out_channels) and (stride == 1)
        hidden_dim = in_channels * expansion_factor

        self.conv = nn.Sequential(
            # Слой расширения (expansion)
            nn.Conv2d(in_channels, hidden_dim, kernel_size=1),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),

            # Depthwise свертка
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=kernel_size, stride=stride,
                      padding=kernel_size // 2, groups=hidden_dim),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),

            # Сжатие (projection)
            nn.Conv2d(hidden_dim, out_channels, kernel_size=1),
            nn.BatchNorm2d(out_channels),

        )

    def forward(self, x):
        if self.use_residual:
            return x + self.conv(x)
            print(self.conv(x).shape)
        else:
            return self.conv(x)
            print(self.conv(x).shape)

class Network(nn.Module):
    def __init__(self, num_classes=1):
        super(Network, self).__init__()

        # Первый сверточный слой с измененными параметрами
        self.conv_stem = nn.Conv2d(2, 32, kernel_size=(6, 10), stride=(2, 3), padding=1)# 187, 411
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU6(inplace=True)

        self.blocks = nn.Sequential(
            # MBConvBlock(32, 64, expansion_factor=1, stride=2), #
            MBConvBlock(32, 48, expansion_factor=6, stride=2),# 93, 206
            MBConvBlock(48, 64, expansion_factor=6, stride=2),# 47, 103
            MBConvBlock(64, 64, expansion_factor=6, stride=1), # 47, 103
            MBConvBlock(64, 128, expansion_factor=6, stride=2),# 24, 52
            MBConvBlock(128, 128, expansion_factor=6, stride=1),# 24, 52
            MBConvBlock(128, 256, expansion_factor=6, stride=2),# 12, 26
            MBConvBlock(256, 256, expansion_factor=6, stride=1),# 12, 26
            MBConvBlock(256, 512, expansion_factor=6, stride=2), # 6, 13
            # MBConvBlock(512, 512, expansion_factor=6, stride=2) # 3, 7
            nn.Conv2d(512, 512, kernel_size= 3, stride= 2, groups= 512), #2,6
            nn.Conv2d(512, 512, kernel_size= 1, stride= 1) #2,6

        )

        # # Глобальный Average Pooling
        # self.global_pool = nn.AdaptiveAvgPool2d(1)

        # Полносвязный слой для классификации
        self.fc1 = nn.Linear(512*12, 16)
        self.relu_fc = nn.ReLU(inplace=True)
        self.dropout_fc = nn.Dropout(p= 0.2) # Dropout после первого FC слоя
        self.fc2 = nn.Linear(16, 1)

        self._initialize_weights()

    def forward(self, image1, image2):
        x = torch.cat((image1, image2), dim=1)
        x = self.relu1(self.bn1(self.conv_stem(x)))
        x = self.blocks(x)
        # x = self.relu2(self.bn2(self.conv_head(x)))
        # x = self.global_pool(x)
        x = torch.flatten(x, 1)
        # x = self.dropout_fc(self.relu_fc(self.fc1(x)))
        x = self.relu_fc(self.fc1(x))
        x = self.fc2(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

Train

In [ ]:
import time

In [ ]:
def train(model, num_epochs, last_epoch, train_loader,
                validation_loader,test_loader,optimizer,device,scheduler):
    start_time = time.time()
    minibatch_loss_list,train_loss_list,valid_loss_list = [] ,[], []
    criterion = torch.nn.MSELoss()

    for epoch in range(last_epoch, num_epochs):
        model.train()
        for batch_idx,((feature1,feature2),targets) in enumerate(train_loader):
            feature1 = feature1.to(device)
            feature2 = feature2.to(device)
            targets = targets.to(device)

            predicted_y = model(feature1,feature2)
            loss = criterion(predicted_y, targets)

            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

            minibatch_loss_list.append(loss.item())
            if batch_idx %100 == 0:
                print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                      f'| Batch {batch_idx:04d}/{len(train_loader):04d} '
                      f'| Loss: {loss:.4f}'
                      f'| Learning rate: {optimizer.param_groups[0]["lr"]}')
        scheduler.step()
        model.eval()
        with torch.no_grad():
            avg_loss_train, accuracy_train = compute_accuracy(model, train_loader, device)
            avg_loss_val, accuracy_val = compute_accuracy(model, validation_loader, device)
            # scheduler.step( avg_loss_val)
            print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                  f'| Train AVG LOSS: {avg_loss_train: .4f} | Accuracy_train: {accuracy_train: .4f} \n'
                  f'| Validation AVG LOSS: {avg_loss_val: .4f} | Accuracy_val: {accuracy_val: .4f} ')
            train_loss_list.append(avg_loss_train)
            valid_loss_list.append(avg_loss_val)

        # checkpoint = {
        #     'model_state_dict': model.state_dict(),
        #     'optimizer_state_dict': optimizer.state_dict()
        # }

        checkpoint_dict['state_model'] = model.state_dict()
        checkpoint_dict['state_opt'] =  optimizer.state_dict()
        checkpoint_dict[ 'state_scheduler'] = scheduler.state_dict()
        checkpoint_dict['train_loss'] = avg_loss_train
        checkpoint_dict['val_loss'] = avg_loss_val
        checkpoint_dict['train_acc'] = accuracy_train
        checkpoint_dict['val_acc'] =  accuracy_val
        checkpoint_dict['EPOCHS'] = num_epochs
        checkpoint_dict['current_epoch'] = epoch + 1
        checkpoint_dict[ 'learning'] = optimizer.param_groups[0]["lr"]

        path = directory_path + str(epoch + 1) + '_epoch.pth'
        torch.save(checkpoint_dict, path)
        # path = '/content/drive/My Drive/checkpoint_model_3.pth'
        # torch.save(checkpoint, path)

        elapsed = (time.time() - start_time)/60
        print(f'Time elapsed: {elapsed:.2f} min')

    elapsed = (time.time() - start_time)/60
    print(f'Total Training Time: {elapsed:.2f} min')

    avg_loss_test, accuracy_test = compute_accuracy(model, test_loader,device)
    print(f'Test AVG LOSS: {avg_loss_test: .4f} | Accuracy_test: {accuracy_test: .4f}')

    return minibatch_loss_list, train_loss_list, valid_loss_list


MAIN

In [ ]:
BATCH_SIZE = 8
RANDOM_SEED = 123
NUM_EPOCHS = 20
last_epoch = 0
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Використовується пристрій: {DEVICE}")

Використовується пристрій: cuda


In [ ]:
from torch.optim.lr_scheduler import StepLR
set_all_seeds(RANDOM_SEED)

model = Network()
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr= 0.001, weight_decay= 0.01)
train_loader, validation_loader, test_loader = get_data(batch_size = BATCH_SIZE)
scheduler = StepLR(optimizer, step_size= 4, gamma= 0.3)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 3,090,353 trainable parameters


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Загрузка словаря из файла
path = './1_epoch.pth'
checkpoint = torch.load(path)

# # Загрузка состояния модели
# save_check = checkpoint['optimizer_state_dict']
# save_check['lr'] = 0.0003
model.load_state_dict( checkpoint['state_model'] )
optimizer.load_state_dict( checkpoint['state_opt'] )
scheduler.load_state_dict( checkpoint['state_scheduler'] )
last_epoch = checkpoint['current_epoch']
# # Загрузка состояния оптимизатора
# optimizer.load_state_dict(save_check)

<All keys matched successfully>

In [ ]:
minibatch_loss, train_acc, valid_acc = train(model = model,
            num_epochs = NUM_EPOCHS,
            last_epoch = last_epoch,
            train_loader = train_loader,
            validation_loader = validation_loader,
            test_loader = test_loader,
            optimizer = optimizer,
            device = DEVICE,
            scheduler = scheduler)

Epoch: 001/020 | Batch 0000/2550 | Loss: 1.3277| Learning rate: 0.001
Epoch: 001/020 | Batch 0100/2550 | Loss: 0.0819| Learning rate: 0.001
Epoch: 001/020 | Batch 0200/2550 | Loss: 0.0481| Learning rate: 0.001
Epoch: 001/020 | Batch 0300/2550 | Loss: 0.1593| Learning rate: 0.001
Epoch: 001/020 | Batch 0400/2550 | Loss: 0.2007| Learning rate: 0.001
Epoch: 001/020 | Batch 0500/2550 | Loss: 0.1744| Learning rate: 0.001
Epoch: 001/020 | Batch 0600/2550 | Loss: 0.3498| Learning rate: 0.001
Epoch: 001/020 | Batch 0700/2550 | Loss: 0.1204| Learning rate: 0.001
Epoch: 001/020 | Batch 0800/2550 | Loss: 0.2920| Learning rate: 0.001
Epoch: 001/020 | Batch 0900/2550 | Loss: 0.0329| Learning rate: 0.001
Epoch: 001/020 | Batch 1000/2550 | Loss: 0.2238| Learning rate: 0.001
Epoch: 001/020 | Batch 1100/2550 | Loss: 0.1540| Learning rate: 0.001
Epoch: 001/020 | Batch 1200/2550 | Loss: 0.0760| Learning rate: 0.001
Epoch: 001/020 | Batch 1300/2550 | Loss: 0.1441| Learning rate: 0.001
Epoch: 001/020 | Bat

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# scheduler = StepLR(optimizer, step_size=1, gamma=1)
minibatch_loss, train_acc, valid_acc = train(model = model,
            num_epochs = 20,
            last_epoch= 1,
            train_loader = train_loader,
            validation_loader = validation_loader,
            test_loader = test_loader,
            optimizer = optimizer,
            device = DEVICE,
            scheduler = scheduler)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


AttributeError: 'Network' object has no attribute 'fc1'

In [ ]:
minibatch_loss, train_acc, valid_acc = train(model = model,
            num_epochs = 20,
            last_epoch= 2,
            train_loader = train_loader,
            validation_loader = validation_loader,
            test_loader = test_loader,
            optimizer = optimizer,
            device = DEVICE,
            scheduler = scheduler)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch: 003/020 | Batch 0000/2550 | Loss: 0.0806| Learning rate: 0.0003
Epoch: 003/020 | Batch 0100/2550 | Loss: 0.0788| Learning rate: 0.0003
Epoch: 003/020 | Batch 0200/2550 | Loss: 0.0338| Learning rate: 0.0003
Epoch: 003/020 | Batch 0300/2550 | Loss: 0.1559| Learning rate: 0.0003
Epoch: 003/020 | Batch 0400/2550 | Loss: 0.0443| Learning rate: 0.0003
Epoch: 003/020 | Batch 0500/2550 | Loss: 0.1884| Learning rate: 0.0003
Epoch: 003/020 | Batch 0600/2550 | Loss: 0.0161| Learning rate: 0.0003
Epoch: 003/020 | Batch 0700/2550 | Loss: 0.0415| Learning rate: 0.0003
Epoch: 003/020 | Batch 0800/2550 | Loss: 0.0288| Learning rate: 0.0003
Epoch: 003/020 | Batch 0900/2550 | Loss: 0.0450| Learning rate: 0.0003
Epoch: 003/020 | Batch 1000/2550 | Loss: 0.0454| Learning rate: 0.0003
Epoch: 003/020 | Batch 1100/2550 | Loss: 0.0244| Learning rate: 0.0003
Epoch: 003/020 | Batch 1200/2550 | Loss: 0.0311| Learning rate: 0.0003
Epoch: 003/020 | Batch 1300/2550 | Loss: 0.0623| Learning rate: 0.0003
Epoch:

RuntimeError: Parent directory /content/drive/My Drive/model_3 does not exist.

In [ ]:
minibatch_loss, train_acc, valid_acc = train(model = model,
            num_epochs = 20,
            last_epoch= 3,
            train_loader = train_loader,
            validation_loader = validation_loader,
            test_loader = test_loader,
            optimizer = optimizer,
            device = DEVICE,
            scheduler = scheduler)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch: 004/020 | Batch 0000/2550 | Loss: 0.0347| Learning rate: 0.0003
Epoch: 004/020 | Batch 0100/2550 | Loss: 0.0388| Learning rate: 0.0003
Epoch: 004/020 | Batch 0200/2550 | Loss: 0.0183| Learning rate: 0.0003
Epoch: 004/020 | Batch 0300/2550 | Loss: 0.0252| Learning rate: 0.0003
Epoch: 004/020 | Batch 0400/2550 | Loss: 0.0619| Learning rate: 0.0003
Epoch: 004/020 | Batch 0500/2550 | Loss: 0.0178| Learning rate: 0.0003
Epoch: 004/020 | Batch 0600/2550 | Loss: 0.0264| Learning rate: 0.0003
Epoch: 004/020 | Batch 0700/2550 | Loss: 0.0275| Learning rate: 0.0003
Epoch: 004/020 | Batch 0800/2550 | Loss: 0.0295| Learning rate: 0.0003
Epoch: 004/020 | Batch 0900/2550 | Loss: 0.0397| Learning rate: 0.0003
Epoch: 004/020 | Batch 1000/2550 | Loss: 0.0522| Learning rate: 0.0003
Epoch: 004/020 | Batch 1100/2550 | Loss: 0.0464| Learning rate: 0.0003
Epoch: 004/020 | Batch 1200/2550 | Loss: 0.0921| Learning rate: 0.0003
Epoch: 004/020 | Batch 1300/2550 | Loss: 0.0252| Learning rate: 0.0003
Epoch:

RuntimeError: Parent directory /content/drive/My Drive/model_3 does not exist.

In [ ]:
criterion1 = torch.nn.MSELoss()
def test_single_example(model, data_loader, device):
    # Переводим модель в режим оценки
    model.eval()

    with torch.no_grad():
        # Извлекаем один батч из data_loader
        for (feature1, feature2), targets in data_loader:
            # Переводим данные на нужное устройство (GPU/CPU)
            feature1 = feature1.to(device)
            feature2 = feature2.to(device)
            targets = targets.to(device)

            # Возьмем только первый пример из батча (индекс 0)
            feature1_single = feature1[0].unsqueeze(0)  # Убираем размерность батча (берем первый элемент)
            feature2_single = feature2[0].unsqueeze(0)  # Убираем размерность батча (берем первый элемент)
            target_single = targets[0].unsqueeze(0)    # Убираем размерность батча (берем первый элемент)

            # Пропускаем один пример через модель
            predicted_y = model(feature1_single, feature2_single)
            loss = criterion1(predicted_y, targets)
            print("loss: ",loss)
            # Печатаем предсказанные и настоящие значения
            print(f"Predicted values: {predicted_y}")
            print(f"Actual values: {target_single}")

            # Можно вывести разницу между предсказанными и настоящими значениями
            difference = predicted_y - target_single
            print(f"Difference: {difference}")

            break  # Останавливаем цикл после первого батча

# Пример использования функции
# Допустим, у вас уже есть model, data_loader и device
test_single_example(model, validation_loader, DEVICE)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([8, 12])) that is different to the input size (torch.Size([1, 12])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss:  tensor(0.0022, device='cuda:0')
Predicted values: tensor([[ 0.9607, -0.0090,  0.0123, -0.0979, -0.0094,  0.9581, -0.0131, -0.0852,
         -0.0140, -0.0086,  0.9567,  0.2936]], device='cuda:0')
Actual values: tensor([[ 0.9999, -0.0031,  0.0117,  0.0214,  0.0031,  1.0000,  0.0011, -0.0085,
         -0.0117, -0.0011,  0.9999,  0.2881]], device='cuda:0')
Difference: tensor([[-0.0392, -0.0060,  0.0006, -0.1193, -0.0125, -0.0419, -0.0142, -0.0767,
         -0.0023, -0.0075, -0.0433,  0.0055]], device='cuda:0')


In [ ]:
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}
path = '/content/drive/My Drive/checkpoint_model3_4.pth'
torch.save(checkpoint, path)

In [ ]:
path = '/content/drive/My Drive/checkpoint_model3_4.pth'
# Загрузка словаря из файла
checkpoint = torch.load(path)

# Загрузка состояния модели
model.load_state_dict(checkpoint['model_state_dict'])

# Загрузка состояния оптимизатора
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


In [ ]:
current_lr_after_scheduler = optimizer.param_groups[0]['lr']
print(current_lr_after_scheduler)

3e-06


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 975,249 trainable parameters
